# Gán nhãn từ loại với mô hình Markov ẩn
Chúng ta sẽ sử dụng Python để xây dựng một mô hình gán nhãn dữ liệu bằng HMM và thuật toán Veterbi

Đọc dữ liệu và tổ chức dữ liệu huấn luyện

In [1]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time
 
#download the treebank corpus from nltk
#một kho ngữ liệu văn bản được phân tích cú pháp để chú thích cấu trúc câu cú pháp hoặc ngữ nghĩa
nltk.download('treebank')   
 
#download the universal tagset from nltk
nltk.download('universal_tagset')
 
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
 
#print the first two sentences along with tags
print(nltk_data[:2])

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')]]


In [2]:
#in mỗi từ với thẻ tương ứng cho hai câu đầu tiên
for sent in nltk_data[:2]:
  for tuple in sent:
    print(tuple)

('Pierre', 'NOUN')
('Vinken', 'NOUN')
(',', '.')
('61', 'NUM')
('years', 'NOUN')
('old', 'ADJ')
(',', '.')
('will', 'VERB')
('join', 'VERB')
('the', 'DET')
('board', 'NOUN')
('as', 'ADP')
('a', 'DET')
('nonexecutive', 'ADJ')
('director', 'NOUN')
('Nov.', 'NOUN')
('29', 'NUM')
('.', '.')
('Mr.', 'NOUN')
('Vinken', 'NOUN')
('is', 'VERB')
('chairman', 'NOUN')
('of', 'ADP')
('Elsevier', 'NOUN')
('N.V.', 'NOUN')
(',', '.')
('the', 'DET')
('Dutch', 'NOUN')
('publishing', 'VERB')
('group', 'NOUN')
('.', '.')


In [3]:
# split data into training and validation set in the ratio 80:20
### 1. BEGIN CODE HERE ###
train_set,test_set =train_test_split(nltk_data,train_size=0.80,test_size=0.20,random_state = 101)
### 1. END CODE HERE ###

In [4]:
# create list of train and test tagged words
### 2. BEGIN CODE HERE ###
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]
### 2. BEGIN CODE HERE ###
print(len(train_tagged_words))
print(len(test_tagged_words))

80310
20366


In [5]:
# check some of the tagged words.
train_tagged_words[:5]

[('Drink', 'NOUN'),
 ('Carrier', 'NOUN'),
 ('Competes', 'VERB'),
 ('With', 'ADP'),
 ('Cartons', 'NOUN')]

In [6]:
#use set datatype to check how many unique tags are present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)
 
# check total words in vocabulary
vocab = {word for word,tag in train_tagged_words}
print(len(vocab))

# X: các từ mà vì một lý do nào đó không thể được chỉ định một danh mục phần lời nói thực sự
# ADP: Adposition là một thuật ngữ bao hàm cho các giới từ 
# DET: determiner là những danh từ hoặc cụm danh từ
# CONJ: conjunction- liên từ(and, or, but)
# prt: particle là các từ chức năng phải được kết hợp với một từ hoặc cụm từ khác để truyền đạt ý nghĩa
# PRON: pronoun Đại từ là những từ thay thế cho danh từ hoặc cụm danh từ

12
{'ADP', 'X', 'ADJ', '.', 'PRON', 'CONJ', 'NUM', 'VERB', 'NOUN', 'DET', 'PRT', 'ADV'}
11052


In [7]:
train_tagged_words

[('Drink', 'NOUN'),
 ('Carrier', 'NOUN'),
 ('Competes', 'VERB'),
 ('With', 'ADP'),
 ('Cartons', 'NOUN'),
 ('At', 'ADP'),
 ('last', 'ADJ'),
 ('count', 'NOUN'),
 (',', '.'),
 ('Candela', 'NOUN'),
 ('had', 'VERB'),
 ('sold', 'VERB'),
 ('$', '.'),
 ('4', 'NUM'),
 ('million', 'NUM'),
 ('*U*', 'X'),
 ('of', 'ADP'),
 ('its', 'PRON'),
 ('medical', 'ADJ'),
 ('devices', 'NOUN'),
 ('in', 'ADP'),
 ('Japan', 'NOUN'),
 ('.', '.'),
 ('Mrs.', 'NOUN'),
 ('Hills', 'NOUN'),
 ('lauded', 'VERB'),
 ('South', 'NOUN'),
 ('Korea', 'NOUN'),
 ('for', 'ADP'),
 ('*-1', 'X'),
 ('creating', 'VERB'),
 ('an', 'DET'),
 ('intellectual-property', 'ADJ'),
 ('task', 'NOUN'),
 ('force', 'NOUN'),
 ('and', 'CONJ'),
 ('special', 'ADJ'),
 ('enforcement', 'NOUN'),
 ('teams', 'NOUN'),
 ('of', 'ADP'),
 ('police', 'NOUN'),
 ('officers', 'NOUN'),
 ('and', 'CONJ'),
 ('prosecutors', 'NOUN'),
 ('trained', 'VERB'),
 ('*', 'X'),
 ('to', 'PRT'),
 ('pursue', 'VERB'),
 ('movie', 'NOUN'),
 ('and', 'CONJ'),
 ('book', 'NOUN'),
 ('pirates', 'NO

Tính Emission Probability(xác suất phát xạ) và Transition Probability(xác suất chuyển đổi trạng thái) từ dữ liệu huấn luyện
-xác suất chuyển đổi trạng thái là khả năng xảy ra của một chuỗi cụ thể, ví dụ, khả năng một danh từ được theo sau bởi một động từ,..
-xác suất phát xạ là xác suất để 1 nhãn đi kèm với 1 từ(ví dụ như: nếu là 1 modal verb thì xác suất can là bao nhiêu )


In [8]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    ### 3. BEGIN CODE HERE ###
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)    #số lần xuất hiện của thẻ được truyền vào xuất hiện trong tập train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    #tổng số từ truyền vào xuất hiện dưới dạng thẻ đã truyền vào.
    count_w_given_tag = len(w_given_tag_list)
    ### 3. END CODE HERE ###     
    return (count_w_given_tag, count_tag)

print(word_given_tag('drink', 'NOUN', train_bag = train_tagged_words))
print(word_given_tag('drink', 'VERB', train_bag = train_tagged_words))

(1, 22966)
(2, 10860)


In [9]:
# compute Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

print(t2_given_t1('VERB', 'NOUN', train_bag = train_tagged_words))
print(t2_given_t1('ADV', 'VERB', train_bag = train_tagged_words))

(3425, 22966)
(911, 10860)


In [10]:
# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)
 
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
print(tags_matrix)

[[1.69577319e-02 3.45482156e-02 1.07061505e-01 3.87243740e-02
  6.96026310e-02 1.01240189e-03 6.32751212e-02 8.47886596e-03
  3.23588967e-01 3.20931405e-01 1.26550242e-03 1.45532778e-02]
 [1.42225638e-01 7.57255405e-02 1.76821072e-02 1.60868734e-01
  5.41995019e-02 1.03786280e-02 3.07514891e-03 2.06419379e-01
  6.16951771e-02 5.68902567e-02 1.85085520e-01 2.57543717e-02]
 [8.05825219e-02 2.09708735e-02 6.33009672e-02 6.60194159e-02
  1.94174761e-04 1.68932043e-02 2.17475723e-02 1.14563107e-02
  6.96893215e-01 5.24271838e-03 1.14563107e-02 5.24271838e-03]
 [9.29084867e-02 2.56410260e-02 4.61323895e-02 9.23720598e-02
  6.87694475e-02 6.00793920e-02 7.82104954e-02 8.96899477e-02
  2.18538776e-01 1.72191828e-01 2.78940029e-03 5.25694676e-02]
 [2.23234631e-02 8.83826911e-02 7.06150308e-02 4.19134386e-02
  6.83371304e-03 5.01138950e-03 6.83371304e-03 4.84738052e-01
  2.12756261e-01 9.56719834e-03 1.41230067e-02 3.69020514e-02]
 [5.59824370e-02 9.33040585e-03 1.13611415e-01 3.51262353e-02
  6

In [11]:
# Matrix(i, j) represents P(jth tag after the ith tag)
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

# X: X is used for words that for some reason cannot be assigned a real part-of-speech category. 
#It should be used very restrictively
# ADP: Adposition là một thuật ngữ bao hàm cho các giới từ 
# DET: determiner là những danh từ hoặc cụm danh từ
# CONJ: conjunction- liên từ(and, or, but)
# prt: particle là các từ chức năng phải được kết hợp với một từ hoặc cụm từ khác để truyền đạt ý nghĩa
# PRON: pronoun Đại từ là những từ thay thế cho danh từ hoặc cụm danh từ

,ADP,X,ADJ,.,PRON,CONJ,NUM,VERB,NOUN,DET,PRT,ADV
ADP,0.016958,0.034548,0.107062,0.038724,0.069603,0.001012,0.063275,0.008479,0.323589,0.320931,0.001266,0.014553
X,0.142226,0.075726,0.017682,0.160869,0.054200,0.010379,0.003075,0.206419,0.061695,0.056890,0.185086,0.025754
ADJ,0.080583,0.020971,0.063301,0.066019,0.000194,0.016893,0.021748,0.011456,0.696893,0.005243,0.011456,0.005243
.,0.092908,0.025641,0.046132,0.092372,0.068769,0.060079,0.078210,0.089690,0.218539,0.172192,0.002789,0.052569
PRON,0.022323,0.088383,0.070615,0.041913,0.006834,0.005011,0.006834,0.484738,0.212756,0.009567,0.014123,0.036902
CONJ,0.055982,0.009330,0.113611,0.035126,0.060373,0.000549,0.040615,0.150384,0.349067,0.123491,0.004391,0.057080
NUM,0.037487,0.202428,0.035345,0.119243,0.001428,0.014281,0.184220,0.020707,0.351660,0.003570,0.026062,0.003570
VERB,0.092357,0.215930,0.066390,0.034807,0.035543,0.005433,0.022836,0.167956,0.110589,0.133610,0.030663,0.083886
NOUN,0.176827,0.028825,0.012584,0.240094,0.004659,0.042454,0.009144,0.149134,0.262344,0.013106,0.043935,0.016895
DET,0.009918,0.045134,0.206411,0.017393,0.003306,0.000431,0.022855,0.040247,0.635906,0.006037,0.000287,0.012074


Xây dựng thuật toán Viterbi

In [12]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))   #set: Không có thứ tự, không thể thay đổi, không được phép trùng lặp
    print(T)
     
    for key, word in enumerate(words):
        #khởi tạo danh sách cột xác suất cho một quan sát nhất định
        p = [] 
        for tag in T:
            ### 4. BEGIN CODE HERE ###
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]   #state[-1]: trạng thái trước đó
                # [('Janet', 'NNP'), ('will', 'VERB'), ('back', 'ADV'), ('the', 'DET'), ('bill', 'NOUN')]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
            ### 4. END CODE HERE ###


        ### 5. BEGIN CODE HERE ###
        pmax = max(p)
        # nhận được trạng thái mà xác suất là max
        state_max = T[p.index(pmax)]
        ### 5. END CODE HERE ###

        state.append(state_max)
    return list(zip(words, state))

Test thuật toán Viterbi

In [13]:
# thử nghiệm thuật toán Viterbi trên một số câu mẫu của tập test
random.seed(1234)      #xác định random seed để nhận được các câu giống nhau khi chạy nhiều lần 
 
# choose random 10 numbers
rndom = [random.randint(1,len(test_set)) for x in range(10)]
 
# list of 10 sents on which we test the model
test_run = [test_set[i] for i in rndom]
print(test_run[:1])
 
# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]
print(test_run_base[:5])
 
# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
print(test_tagged_words[:5])

[[('The', 'DET'), ('company', 'NOUN'), ('is', 'VERB'), ('contesting', 'VERB'), ('the', 'DET'), ('fine', 'NOUN'), ('.', '.')]]
[('The', 'DET'), ('company', 'NOUN'), ('is', 'VERB'), ('contesting', 'VERB'), ('the', 'DET')]
['The', 'company', 'is', 'contesting', 'the']


In [14]:
#Here We will only test 10 sentences to check the accuracy
#as testing the whole training set takes huge amount of time
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

['ADP', 'X', 'ADJ', '.', 'PRON', 'CONJ', 'NUM', 'VERB', 'NOUN', 'DET', 'PRT', 'ADV']
Time taken in seconds:  35.84871530532837
Viterbi Algorithm Accuracy:  93.77990430622009


In [15]:
# #Code to test all the test sentences
# #(takes alot of time to run)
# # tagging the test sentences()
# ### 6. BEGIN CODE HERE ###
# test_tagged_words = [tup for sent in test_set for tup in sent]
# test_untagged_words = [tup[0] for sent in test_set for tup in sent]
# test_untagged_words
# ### 6. END CODE HERE ###
 
# start = time.time()
# tagged_seq = Viterbi(test_untagged_words)
# end = time.time()
# difference = end-start
 
# print("Time taken in seconds: ", difference)
 
# # accuracy
# check = [i for i, j in zip(test_tagged_words, test_untagged_words) if i == j] 
 
# accuracy = len(check)/len(tagged_seq)
# print('Viterbi Algorithm Accuracy: ',accuracy*100)

Tăng độ chính xác bằng cách kết hợp thêm rule

In [16]:
#To improve the performance,we specify a rule base tagger for unknown words 
# specify patterns for tagging
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # nouns số nhiều
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                   # nouns
]

#$ :kết thúc của chuỗi nhập
#. :bất kì ký tự nào ngoại trừ ký tự xuống dòng
#* :ký tự trước có thể được lặp lại 0, hoặc nhiều lần
#+ các ký tự trước có thể được lặp lại 1 hoặc nhiều lần
#? :ký tự trước có thể được lặp lại 0 hoặc 1 lần
#/ :Bắt đầu hoặc kết thúc chuỗi regex
#\ :Biểu diễn một kí tự ngay sau nó từ kí tự đặc biệt thành kí tự thường và ngược lại
#^ :Bắt đầu của từ


# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [26]:
rule_based_tagger.tag(["will"])[0][1]

'NOUN'

In [18]:
#modified Viterbi to include rule based tagger in it
def Viterbi_rule_based(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #khởi tạo danh sách cột xác suất cho một quan sát nhất định
        p = [] 
        for tag in T:
            ### 7. BEGIN CODE HERE ###
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            ### 7. END CODE HERE ###

        ### 8. BEGIN CODE HERE ###
        pmax = max(p)

        state_max = rule_based_tagger.tag([word[0]])[0][1]       
        ### 8. END CODE HERE ###
        
        ### 9. BEGIN CODE HERE ###
        if(pmax==0):
            state_max = rule_based_tagger.tag([word[0]])[0][1] # assign based on rule based tagger
        else:
            if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)]                
        ### 9. END CODE HERE ###
        state.append(state_max)
    return list(zip(words, state))

In [19]:
#test accuracy on subset of test data 
start = time.time()
tagged_seq = Viterbi_rule_based(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  35.93308115005493
Viterbi Algorithm Accuracy:  95.69377990430623


In [27]:
#Check how a sentence is tagged by the two POS taggers
#and compare them
test_sent="being can see Marry"
pred_tags_rule=Viterbi_rule_based(test_sent.split())
pred_tags_withoutRules= Viterbi(test_sent.split())
print(pred_tags_rule)
print(pred_tags_withoutRules)

['ADP', 'X', 'ADJ', '.', 'PRON', 'CONJ', 'NUM', 'VERB', 'NOUN', 'DET', 'PRT', 'ADV']
[('being', 'VERB'), ('can', 'VERB'), ('see', 'VERB'), ('Marry', 'NOUN')]
[('being', 'VERB'), ('can', 'VERB'), ('see', 'VERB'), ('Marry', 'ADP')]
